In [2]:
from model import UNET
from utils import load_checkpoint
import torch
%load_ext autoreload
%autoreload 2

DEVICE = 'cuda:0'
model = UNET(in_channels=3, out_channels=1).to(DEVICE)
load_checkpoint(torch.load("add1024_5000epoch.pth"), model)
model.eval()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=> Loading checkpoint


UNET(
  (ups): ModuleList(
    (0): ConvTranspose2d(2048, 1024, kernel_size=(2, 2), stride=(2, 2))
    (1): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(2048, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (2): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
    (3): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(512, 512, kernel_size=(3, 3), stride=(

In [47]:
from utils import RoadDataset, get_transform
from torch.utils.data import DataLoader
import albumentations as A
from tqdm import tqdm
import math

train_dir = "data/train_images"
train_maskdir = "data/train_masks"

dataset = RoadDataset(
    image_dir=train_dir,
    mask_dir=train_maskdir,
    transform=get_transform(train=True),
)

loader = DataLoader(dataset, batch_size=1,
                    pin_memory=True, shuffle=False, num_workers=2)     

p_list, r_list = [0.0 for i in range(len(dataset))], [0.0 for i in range(len(dataset))]
n_valid = [0 for i in range(len(dataset))]
y_pos_ratio = []
for epoch in tqdm(range(50)):
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(loader):
            x = x.to(device=DEVICE)
            y = y.to(device=DEVICE).unsqueeze(1)
            y_pos_ratio.append(y.sum().item() / y.numel())
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            TP = ((preds == 1)*(y==1)).sum()
            FP = ((preds == 1)*(y==0)).sum()
            FN = ((preds == 0)*(y==1)).sum()
            recall = TP/(TP+FN)
            precision = TP/(TP+FP)
            if math.isnan(recall) or math.isnan(precision):
                continue
            p_list[batch_idx] += precision
            r_list[batch_idx] += recall
            n_valid[batch_idx] += 1
p_list = [p_list[i] / n_valid[i] for i in range(len(dataset))]
r_list = [r_list[i] / n_valid[i] for i in range(len(dataset))]
f1_list = [2*r_list[i]*p_list[i]/(r_list[i]+p_list[i]) for i in range(len(p_list))]

100%|██████████| 50/50 [02:29<00:00,  2.99s/it]


In [63]:
for i in range(len(f1_list)):
    if f1_list[i].item() < 0.989:
        # print(i, f1_list[i].item())
        print(f'{i}, ', end='')

10, 18, 20, 22, 26, 27, 28, 29, 30, 31, 32, 40, 52, 74, 76, 77, 85, 87, 90, 91, 

In [57]:
for i in range(len(f1_list)):
    if p_list[i].item() < 0.99:
        print(i, r_list[i].item())

9 0.9904438853263855
10 0.9087799787521362
17 0.9892675280570984
20 0.8747090101242065
22 0.9857665300369263
26 0.9776500463485718
27 0.9643473625183105
29 0.9784762859344482
30 0.9832307696342468
31 0.9860295653343201
32 0.968249499797821
43 0.9902804493904114
58 0.991472601890564
60 0.989358127117157
64 0.9899277091026306
74 0.9859753847122192
76 0.9255867600440979
77 0.9745124578475952
87 0.9839822053909302
90 0.9850777387619019
91 0.9420167803764343


In [67]:
sum(y_pos_ratio) / len(y_pos_ratio)

0.20964930747922517